In [1]:
!wget https://raw.githubusercontent.com/sberbank-ai/ru-gpts/master/generate_transformers.py
!pip3 install urllib3==1.25.4
!pip3 install transformers==2.8.0
!pip install flask-ngrok

--2020-11-28 13:09:44--  https://raw.githubusercontent.com/sberbank-ai/ru-gpts/master/generate_transformers.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10474 (10K) [text/plain]
Saving to: ‘generate_transformers.py.2’

generate_transforme 100%[===================>]  10.23K  --.-KB/s    in 0s      

2020-11-28 13:09:44 (53.8 MB/s) - ‘generate_transformers.py.2’ saved [10474/10474]



In [2]:
"""
!python generate_transformers.py \
    --model_type=gpt2 \
    --model_name_or_path=sberbank-ai/rugpt3large_based_on_gpt2 \
    --k=5 \
    --p=0.95 \
    --length=100
"""

'\n!python generate_transformers.py     --model_type=gpt2     --model_name_or_path=sberbank-ai/rugpt3large_based_on_gpt2     --k=5     --p=0.95     --length=100\n'

In [ ]:
import os
import argparse
import logging
import numpy as np
import torch

from transformers import (
    CTRLLMHeadModel,
    CTRLTokenizer,
    GPT2LMHeadModel,
    GPT2Tokenizer,
    OpenAIGPTLMHeadModel,
    OpenAIGPTTokenizer,
    TransfoXLLMHeadModel,
    TransfoXLTokenizer,
    XLMTokenizer,
    XLMWithLMHeadModel,
    XLNetLMHeadModel,
    XLNetTokenizer,
)

from flask import Flask, request
from flask_ngrok import run_with_ngrok

#logging.basicConfig(format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s", datefmt="%m/%d/%Y %H:%M:%S", level=logging.INFO,)
#logger = logging.getLogger(__name__)

MAX_LENGTH = int(10000) 

MODEL_CLASSES = {
    "gpt2": (GPT2LMHeadModel, GPT2Tokenizer),
    "ctrl": (CTRLLMHeadModel, CTRLTokenizer),
    "openai-gpt": (OpenAIGPTLMHeadModel, OpenAIGPTTokenizer),
    "xlnet": (XLNetLMHeadModel, XLNetTokenizer),
    "transfo-xl": (TransfoXLLMHeadModel, TransfoXLTokenizer),
    "xlm": (XLMWithLMHeadModel, XLMTokenizer),
}

def set_seed():
    np.random.seed(seed)
    torch.manual_seed(seed)
    if n_gpu > 0:
        torch.cuda.manual_seed_all(seed)

def prepare_ctrl_input(_, tokenizer, prompt_text):
    if temperature > 0.7:
        logger.info("CTRL typically works better with lower temperatures (and lower top_k).")

    encoded_prompt = tokenizer.encode(prompt_text, add_special_tokens=False)
    if not any(encoded_prompt[0] == x for x in tokenizer.control_codes.values()):
        logger.info("WARNING! You are not starting your generation from a control code so you won't get good results")
    return prompt_text


def prepare_xlm_input(model, tokenizer, prompt_text):
    use_lang_emb = hasattr(model.config, "use_lang_emb") and model.config.use_lang_emb
    if hasattr(model.config, "lang2id") and use_lang_emb:
        available_languages = model.config.lang2id.keys()
        if xlm_language in available_languages:
            language = xlm_language
        else:
            language = None
            while language not in available_languages:
                language = input("Using XLM. Select language in " + str(list(available_languages)) + " >>> ")

        model.config.lang_id = model.config.lang2id[language]
    return prompt_text


def prepare_xlnet_input(_, tokenizer, prompt_text):
    prompt_text = (padding_text if padding_text else PADDING_TEXT) + prompt_text
    return prompt_text


def prepare_transfoxl_input(_, tokenizer, prompt_text):
    prompt_text = (padding_text if padding_text else PADDING_TEXT) + prompt_text
    return prompt_text


PREPROCESSING_FUNCTIONS = {
    "ctrl": prepare_ctrl_input,
    "xlm": prepare_xlm_input,
    "xlnet": prepare_xlnet_input,
    "transfo-xl": prepare_transfoxl_input,
}


def adjust_length_to_model(length, max_sequence_length):
    if length < 0 and max_sequence_length > 0:
        length = max_sequence_length
    elif 0 < max_sequence_length < length:
        length = max_sequence_length 
    elif length < 0:
        length = MAX_LENGTH
    return length


model_type="gpt2"
model_name_or_path="sberbank-ai/rugpt3large_based_on_gpt2"
k=5
p=0.95
length=100
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
temperature = 1.0
seed = 42
n_gpu = torch.cuda.device_count()
xlm_language = ""
padding_text = ""
stop_token = "</s>"
num_return_sequences = 1
repetition_penalty = 1.0


model_type = model_type.lower()
model_class, tokenizer_class = MODEL_CLASSES[model_type]
tokenizer = tokenizer_class.from_pretrained(model_name_or_path)
model = model_class.from_pretrained(model_name_or_path)
model.to(device)

length = adjust_length_to_model(length, max_sequence_length=model.config.max_position_embeddings)
#logger.info(args)


app = Flask(__name__)
run_with_ngrok(app)  
@app.route("/")
def main():

    generated_sequences = []
    prompt_text = request.args.get("prompt_text")
    #prompt_text = "%20".join(prompt_text.split("%20")[:20])
    print(prompt_text.split(" ")[:20])

    requires_preprocessing = model_type in PREPROCESSING_FUNCTIONS.keys()
    if requires_preprocessing:
        prepare_input = PREPROCESSING_FUNCTIONS.get(model_type)
        preprocessed_prompt_text = prepare_input(model, tokenizer, prompt_text)
        encoded_prompt = tokenizer.encode(preprocessed_prompt_text, add_special_tokens=False, return_tensors="pt", add_space_before_punct_symbol=True)
    else:
        encoded_prompt = tokenizer.encode(prompt_text, add_special_tokens=False, return_tensors="pt")
    encoded_prompt = encoded_prompt.to(device)

    output_sequences = model.generate(
            input_ids=encoded_prompt,
            max_length=length + len(encoded_prompt[0]),
            temperature=temperature,
            top_k=k,
            top_p=p,
            repetition_penalty=repetition_penalty,
            do_sample=True,
            num_return_sequences=num_return_sequences,
        )

    if len(output_sequences.shape) > 2:
        output_sequences.squeeze_()

    for generated_sequence_idx, generated_sequence in enumerate(output_sequences):
        print("ruGPT:".format(generated_sequence_idx + 1))
        generated_sequence = generated_sequence.tolist()
        text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True)
        text = text[: text.find(stop_token) if stop_token else None]
        total_sequence = (prompt_text + text[len(tokenizer.decode(encoded_prompt[0], clean_up_tokenization_spaces=True)) :])
        generated_sequences.append(total_sequence)

    return " ".join(" ".join(generated_sequences).split()[:50])


if __name__ == '__main__':
    app.run()